
1) Необходимо собрать информацию о вакансиях на вводимую должность
(используем input или через аргументы)
с сайта superjob.ru. Приложение должно анализировать несколько страниц сайта
(также вводим через input или аргументы). 

In [1]:
import numpy as np
import pandas as pd
#from pprint import pprint
import requests
import re
from bs4 import BeautifulSoup as bs

In [2]:
# functions: choosing the job and getting the link to search vacancies by job's name

def get_link():
    link='https://www.superjob.ru/vacancy/search/?keywords=Программист'
    return link

# getting parsed html code
def parsing_page(link = get_link()):
    response = requests.get(link)
    site_text = response.text
    parsed_page = bs(site_text, 'html.parser')
    return parsed_page
    
# getting an iterable object of 30 vacancies from the site's page
def searching_vacancy_block(parsed_site):
    vacancy_blocks = parsed_site.findAll('div', attrs={'class': '_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'})
    return vacancy_blocks

# getting a link to the next page of vacancies from the parsed html code
def searching_link(parsed_page):
    searching_link = parsed_page.find('a', rel='next')
    link_2 = searching_link['href']
    new_link = f'https://www.superjob.ru{link_2}'
    return new_link

# extracting data on each vacancy from an object of 30 vacancies and restructuring it to pandas.dataframe
def extracting_vacancies(vacancy_block):
    vacancy_name = []
    vacancy_link = []
    vacancy_salary = []
    vacancy_resource =[]
    for vacancy in vacancy_block:
        # extracting vacancy's link
        link = vacancy.find('a', attrs={'href': re.compile(r'^/vakansii/')})
        l = 'https://www.superjob.ru'+link['href']

        # storing it into a list variable
        vacancy_link.append(l)
        # extracting vacancy's name
        job_pos = vacancy.find('div', attrs={'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
        j = job_pos.get_text()
        # storing it into a list variable
        vacancy_name.append(j)
        # extracting the salary offered
        job_sal = vacancy.find('span', attrs={'class': re.compile("salary")})
        s = job_sal.get_text()
        s = s.replace('\xa0', ' ')
        # storing it into a list variable
        vacancy_salary.append(s)

        # lists of data into pandas.dataframe
        vacancy_resource.append("SuperJob")
        
    vacancy_df = pd.DataFrame({'Название вакансии': vacancy_name, 'Ссылка на вакансию': vacancy_link, 'Заработная плата': vacancy_salary, 'Источник вакансии': vacancy_resource},
                     columns=['Название вакансии', 'Ссылка на вакансию', 'Заработная плата', 'Источник вакансии'])
    return vacancy_df



In [3]:
# forming an empty dataframe
vacancies_df_sj = pd.DataFrame({'Название вакансии': [], 'Ссылка на вакансию': [], 'Заработная плата': [], 'Источник вакансии': []},
                     columns=['Название вакансии', 'Ссылка на вакансию', 'Заработная плата', 'Источник вакансии'])

# calling function to get a parsed first page with vacancies
parsed_page = parsing_page()

# calling a function to get an object with 30 vacancies from the parsed page
vacancy_block = searching_vacancy_block(parsed_page)


In [4]:
# a cicle of collecting data from the first page, getting a link to the next page and collecting data from it too
while parsed_page.find('a', rel='next'):
    vacancy_df = extracting_vacancies(vacancy_block)
    # appending formed dataframes from each page until there is no "next" page to parse
    vacancies_df_sj = vacancies_df_sj.append(vacancy_df)

    parsed_page = parsing_page(searching_link(parsed_page))
    vacancy_block = searching_vacancy_block(parsed_page)
    #limit to number of vacancies => limiting parsing process
    if len(vacancies_df_sj)>70:
        break

        

In [5]:
vacancies_df_sj.head()

,Название вакансии,Ссылка на вакансию,Заработная плата,Источник вакансии
0,Программист 1С,https://www.superjob.ru/vakansii/programmist-1...,По договорённости,SuperJob
1,Инженер-программист 1 категории отдела внедрен...,https://www.superjob.ru/vakansii/inzhener-prog...,По договорённости,SuperJob
2,Программист 1С (Middle),https://www.superjob.ru/vakansii/programmist-1...,По договорённости,SuperJob
3,Инженер-программист микроконтроллеров,https://www.superjob.ru/vakansii/inzhener-prog...,45 000 — 75 000 ₽,SuperJob
4,Веб-программист,https://www.superjob.ru/vakansii/veb-programmi...,По договорённости,SuperJob


In [6]:
vacancies_df_sj.to_csv('D:/AI/vacancies_superjob.csv', index=False)